---
# 260113_unp_06_main-quest.ipynb
# 6. 프로젝트: 신용거래 이상탐지 데이터 다루기
---
## 프로젝트 수행 방법
- 이번 프로젝트에서 사용할 데이터 이름은 fraud.csv입니다.
신용카드 이상감지 모델을 만들기 위한 데이터이며, 
is_fraud가 사기거래 여부를 나타내는 변수입니다.
이전 노드에서 배운 내용을 모두 동원하여, 
최종적으로 사기거래 여부를 예측하기 위한 데이터를 준비해보세요.
---

---
## 데이터 불러오기, 데이터 확인
---

In [313]:
import pandas as pd
import numpy as np
import seaborn as sns
cc_df = pd.read_csv('/aiffel/data/fraud.csv')
pd.set_option('display.max_columns', 50)

In [314]:
# cc_df.head() 를 통해 컬럼별 개략적 정보 확인 
    # 중복 데이터 있음 : trans_date_trans_time, unix_time
    # 불필요할 것으로 보이는 데이터 있음 : trans_num, city_pop,zip 등 
    
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [315]:
# cc_df.info() 를 통해 데이터 정보 확인 
    # 결측치 없음
    # 각 컬럼별 데이터 타입 확인 

cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [309]:
# cc_df.describe() 를 통해 데이터 통계값 및 이상치 확인
    # amt(구매 금액)은 금융 범죄건이 생기면 이상치가 있을 것이므로 삭제하면 안됨
    # cc_num(신용카드 번호), zip(우편번호) 등은 통계치 검토의미 없음 

cc_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000


---
## 불필요한 컬럼 제거하기
---

In [316]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [317]:
# 텍스트로 구성된 컬럼은 
    # nunique() 값이 너무 크고
    # 구매 상점 이름,카드 소지자 이름 등 상식적으로 사기와 관련되어 있다고 보기 어려운 컬럼 drop
# 주소는 city state, zip, lat, long 이 정보가 중복되므로, lat, long 만 살리고 drop
# trans_num(거래 id) 또한 고유값이므로 사기를 탐지하는데 도움이 되지 않으므로 drop
# unix_time 컬럼은 trans_date_trans_time 컬럼과 중복되므로 drop 
cc_df = cc_df.drop(['merchant','first','last','city','state','zip','job','trans_num','unix_time'], axis=1)
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,2481 Mills Lock,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0


---
## 구매금액의 z-score 계산하기
---

In [318]:
# 고객 행동패턴 확인을 위해 cc_num 기준으로 데이터 정렬
cc_df.sort_values('cc_num')

,trans_date_trans_time,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
378075,2020-08-05 17:03:19,503874407318,shopping_pos,7.77,M,4130 Tiffany Glen Apt. 562,29.5894,-98.5201,1595797,1975-12-28,30.014539,-98.426767,0
230588,2019-12-20 22:21:36,503874407318,health_fitness,72.06,M,4130 Tiffany Glen Apt. 562,29.5894,-98.5201,1595797,1975-12-28,29.260487,-97.961746,0
421413,2020-10-10 12:39:32,503874407318,misc_pos,4.78,M,4130 Tiffany Glen Apt. 562,29.5894,-98.5201,1595797,1975-12-28,29.116928,-99.258936,0
468378,2020-12-13 15:55:44,503874407318,kids_pets,84.56,M,4130 Tiffany Glen Apt. 562,29.5894,-98.5201,1595797,1975-12-28,29.994537,-98.441297,0
345085,2020-06-22 23:52:06,503874407318,entertainment,24.33,M,4130 Tiffany Glen Apt. 562,29.5894,-98.5201,1595797,1975-12-28,29.277168,-98.239289,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53631,2019-04-14 16:57:31,4956828990005111019,entertainment,27.41,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,39.888460,-74.151440,0
485223,2020-12-27 14:14:40,4956828990005111019,home,28.52,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,41.068207,-75.015316,0
264968,2020-02-15 13:32:48,4956828990005111019,shopping_pos,2.95,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,41.435438,-73.839008,0
63475,2019-04-30 17:45:09,4956828990005111019,shopping_pos,7.28,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,41.149072,-75.143152,0


In [319]:
# z-score 계산의 실현가능성 확인을 위해 cc_num 컬럼 value_counts() 확인 
# 각 cc_num 별 데이터의 숫자가 충분하므로 z-score 계산 가능
cc_df['cc_num'].value_counts()

6538441737335434    4392
30270432095985      4392
4364010865167176    4386
4642255475285942    4386
6538891242532018    4386
                    ... 
36913587729122      3641
4681601008538160    3638
30551643947183      3638
4005676619255478    3638
3511378610369890    3628
Name: cc_num, Length: 124, dtype: int64

### z-score 계산 - cc_num 기준(amt_z)

In [320]:
# cc_num 기준으로 z-score 계산을 위해 cc_num 기준 평균, 표준편자 값을 amt_info 로 저장
amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean','std']).reset_index()

# amt_info와 cc_df 테이블 병합 (left join)
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')


# cc_num 기준 z-score 계산
cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']
cc_df[cc_df['is_fraud']==1] # is_fraud =True 값으로 결과 확인

# 불필요한 컬럼 정리
cc_df.drop(['mean','std'], axis=1, inplace=True)
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,2481 Mills Lock,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828


###  z-score 계산 - cc_num + category 기준 

In [321]:
# z-score 계산을 위해 'cc_num','category' 컬럼 기준으로 'mean', 'std' 계산
cat_info = cc_df.groupby(['cc_num','category'])['amt'].agg(['mean', 'std']).reset_index()

# cat_info와 cc_df 테이블 병합 (left join)
cc_df = cc_df.merge(cat_info, on = ['cc_num', 'category'], how = 'left')

# cc_num + category 기준 z-score 계산
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']
cc_df.drop(['mean','std'], axis=1, inplace=True)
cc_df

,trans_date_trans_time,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,2481 Mills Lock,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,594 Berry Lights Apt. 392,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,1.450980
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,4664 Sanchez Common Suite 930,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.128877
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,572 Davis Mountains,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,1.163822
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,144 Evans Islands Apt. 683,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,0.440137


---
## 결제 시간 관련 feature 분석
___

In [322]:
# trans_date_trans_time 컬럼 데이터 타입 변경 후 시간정보 추출 
cc_df['hour'] = pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,2481 Mills Lock,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0


In [323]:
# 시간 정보를 범주로 나눠줌
def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

# 함수 적용하여 시간정보 컬럼 추가 
# value_counts() 결과에 따라 카테고리 기준 조정 가능
cc_df['hour_cat'] = cc_df['hour'].apply(hour_func)
cc_df['hour_cat'].value_counts()

afternoon    176801
night        146697
evening       98662
morning       68974
Name: hour_cat, dtype: int64

In [324]:
# 카드넘버당 시간정보 별 결제 비율 계산
all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index() # 전체 결제건 
hour_cnt = cc_df.groupby(['cc_num','hour_cat'])['amt'].count().reset_index() # 시간별 결제건

# 카드넘버당 시간정보
hour_cnt = hour_cnt.merge(all_cnt, on = 'cc_num', how = 'left')

#컬럼명 수정
hour_cnt = hour_cnt.rename({'amt_x':'hour_cnt' , 'amt_y':'total_cnt'}, axis=1)

# hour_perc 계산
hour_cnt['hour_perc'] = hour_cnt['hour_cnt']/hour_cnt['total_cnt']


# hour_cnt 테이블 정리 후 cc_df와 병합
# 병합시에 on 설정 시 cc_num + hour_cat 추가해서 잡아줘야 함 
hour_cnt = hour_cnt[['cc_num','hour_cat','hour_perc']]
cc_df = cc_df.merge(hour_cnt, on = ['cc_num', 'hour_cat'], how = 'left')

# 불필요한 컬럼 정리
cc_df.drop(['trans_date_trans_time','hour','hour_cat'], axis=1, inplace=True)
cc_df.head()

,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc
0,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470
1,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383
2,180048185037117,grocery_net,46.28,F,2481 Mills Lock,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252
3,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600
4,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882


---
## 거리 관련 feature 분석
___

In [325]:
# 거리 계산 라이브러리 설치 및 임포트
!pip install geopy
from geopy.distance import distance
from datetime import datetime

In [327]:
# 고객과 상점의 위도 경도 컬럼을 활용하여 거리 계산 (연산시간 측정 코드 추가)
start_time = datetime.now()
cc_df['distance'] = cc_df.apply(lambda x : distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
datetime.now() - start_time 

datetime.timedelta(seconds=91, microseconds=832264)

In [328]:
cc_df.head()

,cc_num,category,amt,gender,street,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance
0,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618
1,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605
2,180048185037117,grocery_net,46.28,F,2481 Mills Lock,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167
3,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745
4,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244


In [329]:
# cc_num 기준으로 그루핑하여 distance의 평균, 표준편차 계산
dist_info = cc_df.groupby('cc_num')['distance'].agg(['mean', 'std']).reset_index()

# cc_df 와 dist_info 병합 
cc_df = cc_df.merge(dist_info, on ='cc_num', how = 'left')

In [330]:
# dist_info 사용하여 거리기준 z-score 계산
cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean']) / cc_df['std']

# 계산에 사용된 불필요 컬럼 제거 
cc_df.drop(['lat','long','merch_lat','merch_long','mean','std'], axis=1, inplace = True)
cc_df

,cc_num,category,amt,gender,street,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,149,1978-06-21,0,0.321302,0.317631,0.196470,30.216618,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,124967,1980-12-21,0,-0.114637,-0.652060,0.214383,84.714605,0.345370
2,180048185037117,grocery_net,46.28,F,2481 Mills Lock,71485,1974-07-19,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693
3,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,31515,1971-11-05,0,-0.001304,0.211780,0.213600,22.322745,-1.824486
4,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,378909,1977-02-22,0,-0.393828,-0.575698,0.202882,79.398244,0.002138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,594 Berry Lights Apt. 392,186140,1983-02-08,0,0.360311,1.450980,0.209201,98.043837,0.710346
491130,213112402583773,shopping_pos,25.49,F,4664 Sanchez Common Suite 930,1523,1984-06-03,0,-0.173577,-0.128877,0.207534,127.240424,1.663843
491131,3556613125071656,kids_pets,111.84,M,572 Davis Mountains,28739,1999-12-27,0,0.364619,1.163822,0.160339,100.023736,0.693848
491132,6011724471098086,kids_pets,86.88,F,144 Evans Islands Apt. 683,3684,1981-11-29,0,-0.015209,0.440137,0.218022,80.887812,0.288293


---
## 나이 feature 만들어보기
---

In [333]:
# 생년월일(bod) 컬럼의 데이터 타입을 변경해서 년도 정보만 추출
cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

In [334]:
cc_df

,cc_num,category,amt,gender,street,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,630423337322,grocery_pos,107.23,F,43039 Riley Greens Suite 393,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,269 Sanchez Rapids,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370
2,180048185037117,grocery_net,46.28,F,2481 Mills Lock,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693
3,374930071163758,grocery_pos,64.09,M,61390 Hayes Port,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486
4,2712209726293386,misc_pos,25.58,F,50872 Alex Plain Suite 088,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,594 Berry Lights Apt. 392,186140,1983,0,0.360311,1.450980,0.209201,98.043837,0.710346
491130,213112402583773,shopping_pos,25.49,F,4664 Sanchez Common Suite 930,1523,1984,0,-0.173577,-0.128877,0.207534,127.240424,1.663843
491131,3556613125071656,kids_pets,111.84,M,572 Davis Mountains,28739,1999,0,0.364619,1.163822,0.160339,100.023736,0.693848
491132,6011724471098086,kids_pets,86.88,F,144 Evans Islands Apt. 683,3684,1981,0,-0.015209,0.440137,0.218022,80.887812,0.288293


---
## 범주형 데이터의 one-hot encoding
---

In [335]:
# 원핫인코딩을 위해 컬럼의 nunique값을 확인 
cc_df.nunique()

cc_num          124
category         14
amt           36446
gender            2
street          124
city_pop        122
dob              33
is_fraud          2
amt_z        397188
cat_amt_z    473030
hour_perc       490
distance     491134
dist_z       491134
dtype: int64

In [339]:
# street 컬럼이 남아 있으므로 제외하고 인코딩 
cc_df = cc_df.drop(columns=['street'])

In [342]:
cc_df = pd.get_dummies(cc_df, drop_first=True)

In [343]:
# 원핫인코딩 결과 확인
cc_df.nunique()

cc_num                        124
amt                         36446
city_pop                      122
dob                            33
is_fraud                        2
amt_z                      397188
cat_amt_z                  473030
hour_perc                     490
distance                   491134
dist_z                     491134
category_food_dining            2
category_gas_transport          2
category_grocery_net            2
category_grocery_pos            2
category_health_fitness         2
category_home                   2
category_kids_pets              2
category_misc_net               2
category_misc_pos               2
category_personal_care          2
category_shopping_net           2
category_shopping_pos           2
category_travel                 2
gender_M                        2
dtype: int64

---
## 최종 컬럼 정리 
---

In [344]:
cc_df.drop('cc_num', axis = 1, inplace = True)

In [345]:
cc_df.nunique()

amt                         36446
city_pop                      122
dob                            33
is_fraud                        2
amt_z                      397188
cat_amt_z                  473030
hour_perc                     490
distance                   491134
dist_z                     491134
category_food_dining            2
category_gas_transport          2
category_grocery_net            2
category_grocery_pos            2
category_health_fitness         2
category_home                   2
category_kids_pets              2
category_misc_net               2
category_misc_pos               2
category_personal_care          2
category_shopping_net           2
category_shopping_pos           2
category_travel                 2
gender_M                        2
dtype: int64